<a href="https://colab.research.google.com/github/aysucengiz/ceng463-hw2/blob/task1/task1/CENG463_hw2_task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1
Given a parliamentary speech in one of several languages, identify the ideology of the speaker’s party. In other words, this involves performing binary classification to determine whether the speaker’s party leans left (0) or right (1).


* Select only one country (excluding the UK) and use the parliamentary debates from that country to complete the assigned tasks.

* Fine-tune a **multilingual masked language model:**
  - multilingual BERT
  - XLMRoberta-base
  - language-specific models like Turkish BERT or German BERT.

* In addition, you are required to experiment with a multilingual causal language model:
  - Llama-3.1-8B


https://sidih.si/cdn/121/index.html

In [1]:
!wget -O trainingset-ideology-power.zip https://zenodo.org/records/10450641/files/trainingset-ideology-power.zip?download=1

--2024-12-22 09:41:54--  https://zenodo.org/records/10450641/files/trainingset-ideology-power.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.45.92, 188.185.43.25, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.45.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 813899321 (776M) [application/octet-stream]
Saving to: ‘trainingset-ideology-power.zip’

trainingset-ideolog 100%[===================>] 776.19M  30.3MB/s    in 30s     

2024-12-22 09:42:25 (25.6 MB/s) - ‘trainingset-ideology-power.zip’ saved [813899321/813899321]



In [2]:
!unzip trainingset-ideology-power.zip -d trainingset-ideology-power

Archive:  trainingset-ideology-power.zip
   creating: trainingset-ideology-power/orientation/
  inflating: trainingset-ideology-power/orientation/orientation-at-train.tsv  
  inflating: trainingset-ideology-power/orientation/orientation-ba-train.tsv  
  inflating: trainingset-ideology-power/orientation/orientation-be-train.tsv  
  inflating: trainingset-ideology-power/orientation/orientation-bg-train.tsv  
  inflating: trainingset-ideology-power/orientation/orientation-cz-train.tsv  
  inflating: trainingset-ideology-power/orientation/orientation-dk-train.tsv  
  inflating: trainingset-ideology-power/orientation/orientation-ee-train.tsv  
  inflating: trainingset-ideology-power/orientation/orientation-es-ct-train.tsv  
  inflating: trainingset-ideology-power/orientation/orientation-es-ga-train.tsv  
  inflating: trainingset-ideology-power/orientation/orientation-es-train.tsv  
  inflating: trainingset-ideology-power/orientation/orientation-fi-train.tsv  
  inflating: trainingset-ideolo

## Masked LM

Fine-tune the selected masked language model for each task: For one task use ”text_en” and
for the other task use ”text” (original language).

## Causal LM

For each task, perform inference using the selected causal language model twice: Once using
”text en” and once using ”text” (original language).